# Automated IQM Evaluation of Stream Files


In [ ]:
from automate_evaluation import automate_evaluation

stream_file_folder = "" # Folder with stream file results from indexamajig. 
                        # Note that all stream files in the folder will be processed.

weights_list = [
    (1, 1, 1, 1, 1, 1)
]

"""
Each weight corresponds to one of the six metrics used in calculating the combined IQM value.
The combined IQM is computed by first normalizing each metric across all stream files, then 
multiplying each normalized metric by its assigned weight, and finally summing the results.
The order (or keys) of the weights must match the following metrics:

- 'weighted_rmsd'
- 'fraction_outliers'
- 'length_deviation'
- 'angle_deviation'
- 'peak_ratio'
- 'percentage_indexed'

Multiple weight combinations can be specified if needed.
"""

automate_evaluation(stream_file_folder, weights_list)


Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-511.9_-511.9.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-512.1_-512.0.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-512.0_-511.9.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-512.0_-512.1.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-512.1_-512.1.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-512_-512.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-511.9_-512.0.stream as it appears to have been processed already.
Skipping /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/UOX1_-511.9_-512.1.stream as it appears to have been

Processing chunks in UOX1_-512.1_-512.1.stream: 100%|██████████| 6280/6280 [06:11<00:00, 16.92chunk/s]


Combined metrics CSV written to /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/metric_values_IQM_1_1_1_1_1_1.csv
Best results stream file written to /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/merged_IQM_1_1_1_1_1_1.stream


# Running Partialator for Stream File Merging

In [ ]:
from run_partialator_and_convert import run_partialator_and_convert

stream_file = "/home/bubl3932/files/UOX1/UOX1_original_IQM_v3/merged_IQM_1_1_1_1_1_1.stream" # Stream file to be merged
pointgroup = "mmm"
num_threads = 24
iterations = 1

output_dir = run_partialator_and_convert(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
)

if output_dir is not None:
    print("All done. Results are in:", output_dir)


Running partialator for stream file: /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/merged_IQM_1_1_1_1_1_1.stream


Partialator Progress: 100%|██████████| 3/3 [01:01<00:00, 20.45s/Residuals]

Partialator completed for stream file: /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/merged_IQM_1_1_1_1_1_1.stream
All done. Results are in: /home/bubl3932/files/UOX1/UOX1_original_IQM_v3/merged_IQM_1_1_1_1_1_1_merge_1_iter
